In [2]:
from logger import Logger
logger = Logger()
logger.info("テスト")

テスト


In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
# 訓練データ読み込みorダウンロード
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

In [5]:
# テストデータ読み込みorダウンロード
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [6]:
# データセットからデータloaderを作る
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    logger.info(f"Shape of X [N, C, H, W]: {X.shape}")
    logger.info(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [13]:
# モデル作成
device = ("cuda" if torch.cuda.is_available()
          else "mps" if torch.backends.mps.is_available()
         else "cpu")
logger.info(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
logger.info(model)

Using mps device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [14]:
# 損失関数
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [18]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        # バックプロパゲーション
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1)*len(X)
            logger.info(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [19]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [21]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

loss: 1.151383  [   64/60000]


Epoch 1
-------------------------------


loss: 1.154828  [ 6464/60000]
loss: 0.976347  [12864/60000]
loss: 1.098382  [19264/60000]
loss: 0.979322  [25664/60000]
loss: 1.013232  [32064/60000]
loss: 1.047127  [38464/60000]
loss: 0.985315  [44864/60000]
loss: 1.024441  [51264/60000]
loss: 0.965714  [57664/60000]
loss: 1.034329  [   64/60000]


Test Error: 
 Accuracy: 65.8%, Avg loss: 0.978877 

Epoch 2
-------------------------------


loss: 1.059759  [ 6464/60000]
loss: 0.864877  [12864/60000]
loss: 1.009457  [19264/60000]
loss: 0.894681  [25664/60000]
loss: 0.921324  [32064/60000]
loss: 0.972820  [38464/60000]
loss: 0.913253  [44864/60000]
loss: 0.946820  [51264/60000]
loss: 0.901638  [57664/60000]
loss: 0.948569  [   64/60000]


...

Test Error: 
 Accuracy: 74.5%, Avg loss: 0.717482 

Epoch 9
-------------------------------


loss: 0.780239  [ 6464/60000]
loss: 0.553975  [12864/60000]
loss: 0.764876  [19264/60000]
loss: 0.682637  [25664/60000]
loss: 0.667689  [32064/60000]
loss: 0.751879  [38464/60000]
loss: 0.738568  [44864/60000]
loss: 0.731328  [51264/60000]
loss: 0.704975  [57664/60000]
loss: 0.658944  [   64/60000]


Test Error: 
 Accuracy: 75.3%, Avg loss: 0.699379 

Epoch 10
-------------------------------


loss: 0.756508  [ 6464/60000]
loss: 0.532891  [12864/60000]
loss: 0.748734  [19264/60000]
loss: 0.667557  [25664/60000]
loss: 0.652507  [32064/60000]
loss: 0.732618  [38464/60000]
loss: 0.726607  [44864/60000]
loss: 0.717156  [51264/60000]
loss: 0.689090  [57664/60000]


Test Error: 
 Accuracy: 76.2%, Avg loss: 0.682925 

Done!


In [24]:
# 保存
torch.save(model.state_dict(), "model/model_1.pth")

In [27]:
# ロード
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model/model_1.pth", weights_only=True))

<All keys matched successfully>

In [28]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

In [29]:
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
